In [ ]:
!pip install ply

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
from getpass import getpass
api_key = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [ ]:
%%writefile mylexer.py
import ply.lex as lex

# List of token names
tokens = [
    'BOOLEAN',
    'INTEGER',
    'STRING',
    'UNARY_OP',
    'BINARY_OP',
    'IF',
    'LAMBDA',
    'VARIABLE',
]

# Regular expressions for tokens
t_BOOLEAN = r'T|F'
t_INTEGER = r'I[\!-\~]+'
t_STRING = r'S[\!-\~]*'
t_UNARY_OP = r'U[\!-\~]'
t_BINARY_OP = r'B[\!-\~]'
t_IF = r'\?'
t_LAMBDA = r'L[\!-\~]+'
t_VARIABLE = r'v[\!-\~]+'

# Ignored characters (whitespace)
t_ignore = ' \t'

# Error handling
def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

# Build the lexer
lexer = lex.lex()

Writing mylexer.py


In [ ]:
%%writefile myparser.py
import ply.yacc as yacc
from mylexer import tokens

# # Precedence rules to resolve ambiguities
# precedence = (
#     ('left', 'BINARY_OP'),
#     ('left', 'UNARY_OP'),
# )

# AST Node Classes
class ASTNode:
    pass

class BooleanNode(ASTNode):
    def __init__(self, value):
        self.value = value

class IntegerNode(ASTNode):
    def __init__(self, value):
        self.value = value

class StringNode(ASTNode):
    def __init__(self, value):
        self.value = value

class UnaryOpNode(ASTNode):
    def __init__(self, operator, operand):
        self.operator = operator
        self.operand = operand

class BinaryOpNode(ASTNode):
    def __init__(self, operator, left, right):
        self.operator = operator
        self.left = left
        self.right = right

class IfNode(ASTNode):
    def __init__(self, condition, true_branch, false_branch):
        self.condition = condition
        self.true_branch = true_branch
        self.false_branch = false_branch

class LambdaNode(ASTNode):
    def __init__(self, variable, body):
        self.variable = variable
        self.body = body

class VariableNode(ASTNode):
    def __init__(self, name):
        self.name = name

# Custom base-94 conversion function
def base94_to_int(base94_str):

    value = 0
    for char in base94_str:
        value = value * 94 + ord(char) - ord('!')

    return value

# Parsing rules
def p_program(p):
    'program : expression'
    p[0] = p[1]

def p_expression_boolean(p):
    'expression : BOOLEAN'
    p[0] = BooleanNode(True if p[1] == 'T' else False)

def p_expression_integer(p):
    'expression : INTEGER'
    body = p[1][1:]  # Remove the 'I' indicator
    p[0] = IntegerNode(base94_to_int(body))

def p_expression_string(p):
    'expression : STRING'
    body = p[1][1:]  # Remove the 'S' indicator
    charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!\"#$%&'()*+,-./:;<=>?@[\\]^_`|~ \n"
    p[0] = StringNode(''.join([charset[ord(c) - ord('!')] for c in body]))

def p_expression_unary(p):
    'expression : UNARY_OP expression'
    op = p[1][1]  # Extract the operator character
    p[0] = UnaryOpNode(op, p[2])

def p_expression_binary(p):
    'expression : BINARY_OP expression expression'
    op = p[1][1]  # Extract the operator character
    p[0] = BinaryOpNode(op, p[2], p[3])

def p_expression_if(p):
    'expression : IF expression expression expression'
    p[0] = IfNode(p[2], p[3], p[4])

def p_expression_lambda(p):
    'expression : LAMBDA expression'
    var_id = base94_to_int(p[1][1:])
    p[0] = LambdaNode(var_id, p[2])

def p_expression_variable(p):
    'expression : VARIABLE'
    var_id = base94_to_int(p[1][1:])
    p[0] = VariableNode(var_id)

def p_error(p):
    if p:
        print(f"Syntax error at '{p.value}'")
    else:
        print("Syntax error at EOF")

# Build the parser
parser = yacc.yacc()

# Optional: Function to test the parser and output AST
def test_parser(input_data):
    result = parser.parse(input_data)
    return print_ast(result)

def haskell_map_bin(op):
  if op == '.':
    return '++'
  if op == '=':
    return '=='
  return op

# Function to pretty-print or traverse the AST
def print_ast(node, indent=""):
    if isinstance(node, BooleanNode):
        return "true" if node.value else "false"
    elif isinstance(node, IntegerNode):
        return str(node.value)
    elif isinstance(node, StringNode):
        return f'"{repr(node.value)[1:-1]}"'
    elif isinstance(node, UnaryOpNode):
        return f"({node.operator} {print_ast(node.operand)})"
    elif isinstance(node, BinaryOpNode):
        return f"({print_ast(node.left)} {haskell_map_bin(node.operator)} {print_ast(node.right)})"
    elif isinstance(node, IfNode):
        return f"if ({print_ast(node.condition, indent + '  ')}) then\n" \
               f"{indent} ({print_ast(node.true_branch, indent + '  ')})\n" \
               f"{indent} else \n" \
               f"{indent} ({print_ast(node.false_branch, indent + '  ')})\n" \
               f"{indent}"
    elif isinstance(node, LambdaNode):
        return f"(\\v{node.variable} -> {print_ast(node.body)})"
    elif isinstance(node, VariableNode):
        return f"v{node.name}"

Writing myparser.py


In [ ]:
import importlib  # Needed to reload the module
import mylexer
import myparser

# Reload the module in case you've made changes
importlib.reload(mylexer)
importlib.reload(myparser)

# Test input
# data = """SB%,,/}!.$}7%,#/-%}4/}4(%}M#(//,}/&}4(%}</5.$}P!2)!",%_~~<%&/2%}4!+).'}!}#/523%j}7%}35''%34}4(!4}9/5}(!6%}!},//+}!2/5.$l}S/5e2%}./7},//+).'}!4}4(%}u).$%8wl}N/}02!#4)#%}9/52}#/--5.)#!4)/.}3+),,3j}9/5}#!.}53%}/52}u%#(/w}3%26)#%l}@524(%2-/2%j}4/}+./7}(/7}9/5}!.$}/4(%2}345$%.43}!2%}$/).'j}9/5}#!.},//+}!4}4(%}u3#/2%"/!2$wl~~;&4%2},//+).'}!2/5.$j}9/5}-!9}"%}!$-)44%$}4/}9/52}&)234}#/523%3j}3/}-!+%}352%}4/}#(%#+}4()3}0!'%}&2/-}4)-%}4/}4)-%l}C.}4(%}-%!.4)-%j})&}9/5}7!.4}4/}02!#4)#%}-/2%}!$6!.#%$}#/--5.)#!4)/.}3+),,3j}9/5}-!9}!,3/}4!+%}/52}u,!.'5!'%y4%34wl~"""

# result = myparser.parser.parse(data)
# print(myparser.print_ast(result))

def parse_to_ast(data):
    return myparser.test_parser(data)

print(parse_to_ast("""? B= B$ B$ B$ B$ L$ L$ L$ L# v$ I" I# I$ I% I$ ? B= B$ L$ v$ I+ I+ ? B= BD I$ S4%34 S4 ? B= BT I$ S4%34 S4%3 ? B= B. S4% S34 S4%34 ? U! B& T F ? B& T T ? U! B| F F ? B| F T ? B< U- I$ U- I# ? B> I$ I# ? B= U- I" B% U- I$ I# ? B= I" B% I( I$ ? B= U- I" B/ U- I$ I# ? B= I# B/ I( I$ ? B= I' B* I# I$ ? B= I$ B+ I" I# ? B= U$ I4%34 S4%34 ? B= U# S4%34 I4%34 ? U! F ? B= U- I$ B- I# I& ? B= I$ B- I& I# ? B= S4%34 S4%34 ? B= F F ? B= I$ I$ ? T B. B. SM%,&k#(%#+}IEj}3%.$}z3/,6%},!.'5!'%y4%34} U$ B+ I# B* I$> I1~s:U@ Sz}4/}#,!)-}0/).43}&/2})4 S)&})3}./4}#/22%#4 S").!29}q})3}./4}#/22%#4 S").!29}q})3}./4}#/22%#4 S").!29}q})3}./4}#/22%#4 S").!29}k})3}./4}#/22%#4 S5.!29}k})3}./4}#/22%#4 S5.!29}_})3}./4}#/22%#4 S5.!29}a})3}./4}#/22%#4 S5.!29}b})3}./4}#/22%#4 S").!29}i})3}./4}#/22%#4 S").!29}h})3}./4}#/22%#4 S").!29}m})3}./4}#/22%#4 S").!29}m})3}./4}#/22%#4 S").!29}c})3}./4}#/22%#4 S").!29}c})3}./4}#/22%#4 S").!29}r})3}./4}#/22%#4 S").!29}p})3}./4}#/22%#4 S").!29}{})3}./4}#/22%#4 S").!29}{})3}./4}#/22%#4 S").!29}d})3}./4}#/22%#4 S").!29}d})3}./4}#/22%#4 S").!29}l})3}./4}#/22%#4 S").!29}N})3}./4}#/22%#4 S").!29}>})3}./4}#/22%#4 S!00,)#!4)/.})3}./4}#/22%#4 S!00,)#!4)/.})3}./4}#/22%#4"""))
#parse_to_ast('SY7X3U-X')

if (((((((\v3 -> (\v3 -> (\v3 -> (\v2 -> v3)))) $ 1) $ 2) $ 3) $ 4) == 3)) then
 (if ((((\v3 -> v3) $ 10) == 10)) then
   (if (((3 D "test") == "t")) then
     (if (((3 T "test") == "tes")) then
       (if ((("te" ++ "st") == "test")) then
         (if ((! (true & false))) then
           (if ((true & true)) then
             (if ((! (false | false))) then
               (if ((false | true)) then
                 (if (((- 3) < (- 2))) then
                   (if ((3 > 2)) then
                     (if (((- 1) == ((- 3) % 2))) then
                       (if ((1 == (7 % 3))) then
                         (if (((- 1) == ((- 3) / 2))) then
                           (if ((2 == (7 / 3))) then
                             (if ((6 == (2 * 3))) then
                               (if ((3 == (1 + 2))) then
                                 (if ((($ 15818151) == "test")) then
                                   (if (((# "test") == 15818151)) then
                                     (if ((! false

Generating LALR tables


In [ ]:
from myparser import *
ALPHABET = '''abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!"#$%&'()*+,-./:;<=>?@[\\]^_`|~ \n'''
ALPHABET_MAP = {ch: i for i, ch in enumerate(ALPHABET)}

class EvaluationError(Exception):
    pass

def debug_ast(node):
  print("debug", print_ast(node))
  return node

def eval_ast(node, env=None):
    if env is None:
        env = {}
    # print("evaling", print_ast(node), type(node))
    if isinstance(node, BooleanNode):
        return lambda: node.value
    elif isinstance(node, IntegerNode):
        return lambda: node.value
    elif isinstance(node, StringNode):
        return lambda: node.value
    elif isinstance(node, UnaryOpNode):
        return lambda: evaluate_unary_op(node.operator, eval_ast(node.operand, env))
    elif isinstance(node, BinaryOpNode):
        if node.operator == '$':
            # print("evaling $", print_ast(node.left), '$$$', print_ast(node.right))
            return lambda: eval_ast(node.left, env)()(eval_ast(node.right, env))()
        return lambda: evaluate_binary_op(node.operator, eval_ast(node.left, env), eval_ast(node.right, env))
    elif isinstance(node, IfNode):
        return lambda: evaluate_if(eval_ast(node.condition, env), eval_ast(node.true_branch, env), eval_ast(node.false_branch, env))
    elif isinstance(node, LambdaNode):
        return lambda: lambda arg: eval_ast(node.body, {**env, node.variable: arg})
    elif isinstance(node, VariableNode):
        return lambda: env[node.name]()
        # if node.name in env:
        #     return env[node.name]
        # raise EvaluationError(f"Unbound variable: v{node.name}")
    else:
        raise EvaluationError(f"Unknown node type {node}")

def c_style_mod(x: int, y: int) -> int:
    if y == 0:
        raise ValueError("The modulus divisor cannot be zero.")

    result = x % y

    if (x < 0) != (y < 0) and result != 0:
        result -= y

    return result

def c_style_div(x: int, y: int) -> int:
    if y == 0:
        raise ValueError("The divisor cannot be zero.")

    quotient = x // y
    remainder = x % y

    # Adjust for truncation towards zero
    if (x < 0) != (y < 0) and remainder != 0:
        quotient += 1

    return quotient

def evaluate_unary_op(operator, operand_thunk):
    operand = operand_thunk()
    if operator == '-':
        return -operand
    elif operator == '!':
        return not operand
    elif operator == '#':
        if isinstance(operand, str):
            return base94_to_int(str_to_base94(operand))
        raise EvaluationError("Unary # expects a string")
    elif operator == '$':
        if isinstance(operand, int):
            return base94_to_normal_str(int_to_base94(operand))
        raise EvaluationError("Unary $ expects an integer")
    else:
        raise EvaluationError(f"Unknown unary operator: {operator}")

def evaluate_binary_op(operator, left_thunk, right_thunk):

    if operator == '$':
      raise EvaluationError("Binary $ should be handled before")

    left = left_thunk()
    right = right_thunk()
    if operator == '+':
        return left + right
    elif operator == '-':
        return left - right
    elif operator == '*':
        return left * right
    elif operator == '/':
        if right == 0:
            raise EvaluationError("Division by zero")
        return c_style_div(left, right)
    elif operator == '%':
        return c_style_mod(left, right)
    elif operator == '<':
        return left < right
    elif operator == '>':
        return left > right
    elif operator == '=':
        return left == right
    elif operator == '|':
        return left or right
    elif operator == '&':
        return left and right
    elif operator == '.':
        return left + right  # String concatenation
    elif operator == 'T':
        return right[:left]
    elif operator == 'D':
        return right[left:]
    else:
        raise EvaluationError(f"Unknown binary operator: {operator}")

def evaluate_if(condition_thunk, true_branch_thunk, false_branch_thunk):
    if condition_thunk():
        return true_branch_thunk()
    else:
        return false_branch_thunk()

# Helper functions to convert strings and integers
def int_to_base94(number):

    if number == 0:
        return chr(33)

    base94 = ''
    while number > 0:
        number, remainder = divmod(number, 94)
        base94 = chr(33 + remainder) + base94

    return base94

def str_to_base94(msg):
    return ''.join(chr(33 + ALPHABET_MAP[ch]) for ch in msg)

def base94_to_normal_str(body):
    charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!\"#$%&'()*+,-./:;<=>?@[\\]^_`|~ \n"
    return ''.join([charset[ord(c) - ord('!')] for c in body])

def base94_to_str(base94_number):
    charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!\"#$%&'()*+,-./:;<=>?@[\\]^_`|~ \n"
    result = ""
    for char in str(base94_number):
        result += chr(charset.index(char) + 33)
    return result




In [ ]:
import sys
sys.set_int_max_str_digits(0)

# print(send('get lambdaman10'))
code = send_raw(encode_message('get spaceship22'))
print(code)
print(print_ast(parser.parse(code)))

NameError: name 'send_raw' is not defined

In [ ]:
import json
import jsonpickle

def print_ast_json(node):
  return jsonpickle.encode(node)

def ast_to_dict(node):
  return json.loads(jsonpickle.encode(node))

def dict_to_ast(d):
  return jsonpickle.decode(json.dumps(d))


def ast_to_code(node):
    if isinstance(node, BooleanNode):
        return 'T' if node.value else 'F'
    elif isinstance(node, IntegerNode):
        return 'I' + int_to_base94(node.value)
    elif isinstance(node, StringNode):
        return 'S' + str_to_base94(node.value)
    elif isinstance(node, UnaryOpNode):
        return 'U' + node.operator + ' ' + ast_to_code(node.operand)
    elif isinstance(node, BinaryOpNode):
        return 'B' + node.operator + ' ' + ast_to_code(node.left) + ' ' + ast_to_code(node.right)
    elif isinstance(node, IfNode):
        return '? ' + ast_to_code(node.condition) + ' ' + ast_to_code(node.true_branch) + ' ' + ast_to_code(node.false_branch)
    elif isinstance(node, LambdaNode):
        return 'L' + int_to_base94(node.variable) + ' ' + ast_to_code(node.body)
    elif isinstance(node, VariableNode):
        return 'v' + int_to_base94(node.name)
    else:
        raise ValueError(f"Unknown node type: {type(node)}")



In [ ]:
def with_repeat(body_fn):
  def repeat(body, count):
    return BinaryOpNode("~", BinaryOpNode("~", VariableNode(10), body), IntegerNode(count))

  return dict_to_ast({
        "py/object": "myparser.BinaryOpNode",
        "operator": "$",
        "left": {
            "py/object": "myparser.LambdaNode",
            "variable": 10,
            "body": ast_to_dict(body_fn(repeat))
        },
        "right": {
            "py/object": "myparser.LambdaNode",
            "variable": 1,
            "body": {
                "py/object": "myparser.BinaryOpNode",
                "operator": "$",
                "left": {
                    "py/object": "myparser.LambdaNode",
                    "variable": 1,
                    "body": {
                        "py/object": "myparser.BinaryOpNode",
                        "operator": "$",
                        "left": {
                            "py/object": "myparser.LambdaNode",
                            "variable": 2,
                            "body": {
                                "py/object": "myparser.BinaryOpNode",
                                "operator": "$",
                                "left": {
                                    "py/object": "myparser.VariableNode",
                                    "name": 1
                                },
                                "right": {
                                    "py/object": "myparser.BinaryOpNode",
                                    "operator": "$",
                                    "left": {
                                        "py/object": "myparser.VariableNode",
                                        "name": 2
                                    },
                                    "right": {
                                        "py/object": "myparser.VariableNode",
                                        "name": 2
                                    }
                                }
                            }
                        },
                        "right": {
                            "py/object": "myparser.LambdaNode",
                            "variable": 2,
                            "body": {
                                "py/object": "myparser.BinaryOpNode",
                                "operator": "$",
                                "left": {
                                    "py/object": "myparser.VariableNode",
                                    "name": 1
                                },
                                "right": {
                                    "py/object": "myparser.BinaryOpNode",
                                    "operator": "$",
                                    "left": {
                                        "py/object": "myparser.VariableNode",
                                        "name": 2
                                    },
                                    "right": {
                                        "py/object": "myparser.VariableNode",
                                        "name": 2
                                    }
                                }
                            }
                        }
                    }
                },
                "right": {
                    "py/object": "myparser.LambdaNode",
                    "variable": 3,
                    "body": {
                        "py/object": "myparser.LambdaNode",
                        "variable": 2,
                        "body": {
                            "py/object": "myparser.IfNode",
                            "condition": {
                                "py/object": "myparser.BinaryOpNode",
                                "operator": "=",
                                "left": {
                                    "py/object": "myparser.VariableNode",
                                    "name": 2
                                },
                                "right": {
                                    "py/object": "myparser.IntegerNode",
                                    "value": 1
                                }
                            },
                            "true_branch": {
                                "py/object": "myparser.VariableNode",
                                "name": 1
                            },
                            "false_branch": {
                                "py/object": "myparser.BinaryOpNode",
                                "operator": ".",
                                "left": {
                                    "py/object": "myparser.VariableNode",
                                    "name": 1
                                },
                                "right": {
                                    "py/object": "myparser.BinaryOpNode",
                                    "operator": "$",
                                    "left": {
                                        "py/object": "myparser.VariableNode",
                                        "name": 3
                                    },
                                    "right": {
                                        "py/object": "myparser.BinaryOpNode",
                                        "operator": "-",
                                        "left": {
                                            "py/object": "myparser.VariableNode",
                                            "name": 2
                                        },
                                        "right": {
                                            "py/object": "myparser.IntegerNode",
                                            "value": 1
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    })



In [ ]:
import requests
import time


URL = 'https://boundvariable.space/communicate'
AUTH_HEADER = f'Bearer {api_key}'

ALPHABET = '''abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!"#$%&'()*+,-./:;<=>?@[\\]^_`|~ \n'''
ALPHABET_MAP = {ch: i for i, ch in enumerate(ALPHABET)}


class DecodingError(Exception):
    pass

def decode_string(s: str) -> str:
    if s[0] != 'S':
        raise DecodingError('unknown token')

    return ''.join(ALPHABET[ord(ch) - 33] for ch in s[1:])


def encode_message(msg: str) -> str:
    return 'S' + ''.join(chr(33 + ALPHABET_MAP[ch]) for ch in msg)

def send_raw(data: str, ttl = 1):
  try:
    time.sleep(ttl)
    resp = requests.post(URL, headers={'Authorization': AUTH_HEADER}, data=data)
    resp.raise_for_status()
    return resp.text
  except requests.HTTPError as e:
    print(f"http error {e}");
    send_raw(data, ttl + 1)
    # noinspection PyBroadException


def send(msg: str, tryies = 100):
    data = encode_message(msg)
    text = send_raw(data)
    try:
        return eval_ast(parser.parse(text))()
    except Exception as e:
        print(f"Error decoding response: {e}")
        print("Text:\n" + text)
        print(print_ast(parser.parse(text)))


In [ ]:
code = ast_to_code(with_repeat(lambda repeat:
                                           BinaryOpNode(".",
                                                        StringNode("solve lambdaman8 "),
                                                        repeat(
                                                            BinaryOpNode(".",
                                                              repeat(StringNode("D"), 120),
                                                              BinaryOpNode(".",
                                                                repeat(StringNode("L"), 120),
                                                                BinaryOpNode(".",
                                                                  repeat(StringNode("U"), 120),
                                                                  repeat(StringNode("R"), 120)
                                                                )
                                                              )
                                                            ),
                                                            100
                                                        )
                                           )))
print(print_ast(parser.parse(code)))
decode_string(send_raw(code))

((\v10 -> ("solve lambdaman8 " ++ ((v10 ~ (((v10 ~ "D") ~ 120) ++ (((v10 ~ "L") ~ 120) ++ (((v10 ~ "U") ~ 120) ++ ((v10 ~ "R") ~ 120))))) ~ 100))) $ (\v1 -> ((\v1 -> ((\v2 -> (v1 $ (v2 $ v2))) $ (\v2 -> (v1 $ (v2 $ v2))))) $ (\v3 -> (\v2 -> if ((v2 == 1)) then
 (v1)
 else 
 ((v1 ++ (v3 $ (v2 - 1))))
)))))


'Correct, you solved lambdaman8 with a score of 202!\n'

In [ ]:
from itertools import groupby

def run_length_encode(data: str):
    """Returns run length encoded Tuples for string"""
    # A memory efficient (lazy) and pythonic solution using generators
    return ((x, sum(1 for _ in y)) for x, y in groupby(data))

def limited_rle(data, limit):
  chunks = []
  for ch, ct in run_length_encode(data):
    if ct < limit:
      if chunks and chunks[-1]["type"] == "str":
        chunks[-1]["str"] += ch * ct
      else:
        chunks.append({"type": "str", "str": ch * ct })
    else:
      chunks.append({"type": "group", "char": ch, "count": ct })
  return chunks

def rle_ast(data: str):
  rles = limited_rle(data, 1000)
  def get_body(repeat):
    def get_body_inner(l):
      if (len(l) == 1):
        if (l[0]["type"] == "group"):
          if l[0]["count"] == 1:
            return StringNode(l[0]["char"])
          for i in reversed(range(2, 600)):
            if l[0]["count"] % i == 0:
              return repeat(StringNode(l[0]["char"] * i), l[0]["count"] // i)
          return BinaryOpNode(".", StringNode(l[0]["char"]), get_body_inner([{
              "type": "group",
              "char": l[0]["char"],
              "count": l[0]["count"] - 1 }]))
        return StringNode(l[0]["str"])
      left = l[:len(l)//2]
      right = l[len(l)//2:]
      return BinaryOpNode(".", get_body_inner(left), get_body_inner(right))
    return get_body_inner(rles)
  return with_repeat(get_body)


from pathlib import Path
minif = {}
for i in []:#[23, 24, 25]:
  contents = Path(f"spaceship/spaceship{i}.out").read_text()
  mess = f"solve spaceship{i} " + contents.strip()
  code = encode_message(mess)
  minif[i] = ast_to_code(rle_ast(mess))
  print(f"spaceship{i} {len(code)}->{len(minif[i])}")
  # res = send_raw(minif)
  # print(decode_string(res))
# spaceships24_rle_ast = rle_ast("solve spaceship24 " + spaceships24_sln)
# limited_rle(spaceships20_sln, 200)[:1000]


In [ ]:
print(send('solve spaceship24 ' + Path(f"spaceship/spaceship24.out").read_text()[:1000000]))

NameError: name 'Path' is not defined

In [ ]:
print(decode_string(send_raw(ast_to_code(rle_ast(f'solve spaceship24 {(("6" * 1200) + (("7" * 1200))) * 1001}')))))

Time limit exceeded


In [ ]:
print(send('get spaceship'))

In [ ]:
len(eval_ast(spaceships20_rle_ast)())
# print(len(spaceships20_sln))
# len(ast_to_code(spaceships20_rle_ast))

In [ ]:
#eval_ast(spaceships20_rle_ast)()
print(len(ast_to_code(spaceships24_rle_ast)))
decode_string(send_raw(ast_to_code(spaceships24_rle_ast)))

In [ ]:
print(send('get spaceship'))

In [ ]:
import time

def fetch_concattable_task(task_get_cmd):
  code = send_raw(encode_message(task_get_cmd))
  # print("code parsed to ast", parse_to_ast(code))
  eval_result = send_raw(f"B. {encode_message('echo ')} {code}")
  res = eval_ast(parser.parse(eval_result))()
  print(f"result for {task_get_cmd}: \n{res}")
  return res

lambdamans = []
for i in range(1, 22):
  while True:
    try:
      lambdamans.append(fetch_concattable_task(f"get lambdaman{i}"))
    except requests.HTTPError:
      continue
    break

!mkdir lambdaman

for i, lambdaman in enumerate(lambdamans):
  with open(f'lambdaman/lambdaman{i + 1:02}.txt', 'w') as f:
    f.write(lambdaman)

result for get lambdaman1: 
###.#...
...L..##
.#######


You scored some points for using the echo service!

result for get lambdaman2: 
L...#.
#.#.#.
##....
...###
.##..#
....##


You scored some points for using the echo service!

result for get lambdaman3: 
......
.#....
..#...
...#..
..#L#.
.#...#
......


You scored some points for using the echo service!

result for get lambdaman4: 
#####################
#...#.#.........#...#
#.###.#.#####.###.###
#...#.#.....#.......#
###.#.#.###.#########
#.#....L..#.#.......#
#.#####.###.#.###.###
#.#.#...#.......#...#
#.#.#######.#######.#
#.#...#.#...#.#.....#
#.#.###.#.###.###.#.#
#.....#...#.......#.#
#.###.###.###.#####.#
#.#.#...#...#...#...#
###.#.#.#.#####.###.#
#...#.#...#.....#...#
#.###.#.#.#####.#####
#.....#.#.....#.#...#
#.###.#.#.#.#.#.#.###
#.#...#.#.#.#.#.....#
#####################


You scored some points for using the echo service!

result for get lambdaman5: 
.....########...
....#...........
...#..######....
..#..#......#

In [ ]:
import time

def fetch_concattable_task(task_get_cmd):
  code = send_raw(encode_message(task_get_cmd))
  # print("code parsed to ast", parse_to_ast(code))
  eval_result = send_raw(f"B. {encode_message('echo ')} {code}")
  res = eval_ast(parser.parse(eval_result))()
  print(f"result for {task_get_cmd}: \n{res}")
  return res

spaceships = []
for i in range(1, 25):
  while True:
    try:
      spaceships.append(fetch_concattable_task(f"get spaceship{i}"))
    except requests.HTTPError:
      continue
    break

!mkdir spaceship

for i, spaceship in enumerate(spaceships):
  with open(f'spaceship/spaceship{i + 1:02}.txt', 'w') as f:
    f.write(spaceship)

In [ ]:
import time

def fetch_concattable_task(task_get_cmd):
  code = send_raw(encode_message(task_get_cmd))
  # print("code parsed to ast", parse_to_ast(code))
  eval_result = send_raw(f"B. {encode_message('echo ')} {code}")
  res = eval_ast(parser.parse(eval_result))()
  print(f"result for {task_get_cmd}: \n{res}")
  return res

_3ds = []
for i in range(1, 13):
  while True:
    try:
      _3ds.append(fetch_concattable_task(f"get 3d{i}"))
    except requests.HTTPError:
      continue
    break

!mkdir 3ds

for i, _3ds in enumerate(_3ds):
  with open(f'3ds/3ds{i + 1:02}.txt', 'w') as f:
    f.write(_3ds)

In [ ]:
import time

def fetch_concattable_task(task_get_cmd):
  code = send_raw(encode_message(task_get_cmd))
  # print("code parsed to ast", parse_to_ast(code))
  eval_result = send_raw(f"B. {encode_message('echo ')} {code}")
  res = eval_ast(parser.parse(eval_result))()
  print(f"result for {task_get_cmd}: \n{res}")
  return res

efficiencies = []
for i in range(1, 14):
  while True:
    try:
      efficiencies.append(fetch_concattable_task(f"get efficiency{i}"))
    except requests.HTTPError:
      continue
    break

!mkdir efficiency

for i, efficiencies in enumerate(efficiencies):
  with open(f'efficiency/efficiency{i + 1:02}.txt', 'w') as f:
    f.write(efficiencies)

In [ ]:
from google.colab import files
files.download('lambdaman/lambdaman10.txt')

In [ ]:
print(send("get language_test"))

In [ ]:
lambdaman9_code = send_raw(encode_message("get lambdaman9"))
print(lambdaman9_code)
print(print_ast(parser.parse(lambdaman9_code)))

In [ ]:
print(send("get lambdaman4"))

In [ ]:
lambdaman6_sol = '''B$ L+ B. S3/,6%},!-"$!-!.[} B$ B$ v+ SL IRR L" B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L$ L# ? B= v# I" v" B. v" B$ v$ B- v# I"'''
print(print_ast(parser.parse(lambdaman6_sol)))
# encode_message('solve lambdaman6 ')
decode_string(send_raw(lambdaman6_sol))

In [ ]:
print(send("get lambdaman"))

In [ ]:
print(send('solve lambdaman1 LLLDURRRUDRRURR'))

In [ ]:
print(send('solve lambdaman2 RDURRDDRRUUDDLLLDLLDDRRRUR'))

In [ ]:
print(send('solve lambdaman3 RDURRDDRRUUDDLLLDLLDDRRRUR'))

In [ ]:
print(send('solve spaceship1 31619'))

In [ ]:
print(send('solve spaceship2 8499532511985293271355654363762549151128114253595'))

In [ ]:
print(send(
'''solve 3d1
.  . . 1 . 1 . . 1 .
.  . A - . = . A = .
.  . < . . . . . S .
.  v A * . * S . . .
.  . . . . . . . . .
-1 @ 4 v . . . . . .
.  3 . . . . . . . .
.  . 1 @ 4 . . . . .
.  . . 3 . . . . . .
.  . . . . . . . . .
'''))

In [ ]:
print(send('get /etc/passwd'))

In [ ]:
print(send('login headmaster lambda'))

In [ ]:
print(send(
'''test 3d 1 1
.  . . 1 . 1 . . 1 .
.  . A - . = . A = .
.  . < . . . . . S .
.  v A * . * S . . .
.  . . . . . . . . .
-1 @ 4 v . . . . . .
.  3 . . . . . . . .
.  . 1 @ 4 . . . . .
.  . . 3 . . . . . .
.  . . . . . . . . .
'''))

In [ ]:
print(send('get index'))

In [ ]:
print(send('get spaceship'))

In [ ]:
print(send('get 3d'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(send('get efficiency'))

In [ ]:
with open(f'spaceship/output.txt', 'r') as f:
  for i in f.read().split("\n"):
    print(send(i))

In [ ]:
import math
def encode_number(num):
  line = ""
  while num > 0:
    line = ''.join([chr(33 + num % 94)]) + line
    num = int(num / 94)
  return "I" + line


# raw = """B$ B$ L" B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L$ L# ? B= v# I" v" B. v" B$ v$ B- v# I" S3 """ + encode_number(199)
# print(raw)
# # print(parser.parse(raw))
# print(parse_to_ast(raw))


def fast_zip(msg, inner = 10):
  repeated = (msg[0],0,1)
  best = repeated
  for i in range(1, len(msg)):
    if msg[i] == msg[i - 1]:
      repeated = (repeated[0], repeated[1] , repeated[2]+ 1)
    else:
      repeated = (msg[i], i, 1)
    if (best[2] < repeated[2]):
      best = repeated

  enc = encode_message(msg)
  if (best[2] > 100 and inner > 0):
    (c,s,l) = best
    ss = int(best[2]/10)
    # ss = int(math.sqrt(best[2]))
    rp = """B$ B$ L" B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L$ L# ? B= v# I" v" B. v" B$ v$ B- v# I" """ + encode_message(best[0] * ss) + " " + encode_number(int(best[2]/ss))
    line = rp
    bb = best[1] + best[2] % ss
    if bb > 0:
        line = f"B. {fast_zip(msg[:bb], inner - 1)} {line}"
    if best[1] + best[2] < len(msg):
        line = f"B. {line} {fast_zip(msg[best[1]+best[2]:], inner - 1)}"
    if len(line) > len(enc):
        return enc
    return line
  return enc

def test(line):
  print(f"TEST: {line}")
  raw = fast_zip(line)
  print(raw)
  e = eval_ast(parser.parse(send_raw(f"B. {encode_message('echo ')} {raw}")))().split("\n\n")[0]
  print(e)
  if (e != line):
    print("ERROR")

test("bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb")

TEST: bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
B. S"""""""" B$ B$ L" B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L$ L# ? B= v# I" v" B. v" B$ v$ B- v# I" S"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""" I+
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb

In [ ]:
#for file in ('spaceship23', 'spaceship24', 'spaceship25'):


for file in ['spaceship24']:#, 'spaceship24', 'spaceship25']:
    with open(file + ".out", 'r') as f:
      res = f.read()
      raw = fast_zip(f'solve {file} {res}', 20)
      l = len(raw)
      print(l)
      if (l < 1000000):
        print(eval_ast(parser.parse(send_raw(raw)))())

983916
Time limit exceeded


In [ ]:
'a'*10

'aaaaaaaaaa'